In [1]:
import pandas as pd  
import numpy as np   
from bs4 import BeautifulSoup    
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초.

In [2]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 melon(음원사이트)
driver.get('http://www.melon.com')
time.sleep(0.5)

In [138]:
driver.find_element_by_css_selector(".menu_bg.menu04").click() #멜론DJ 클릭
time.sleep(0.5)
driver.find_element_by_css_selector(".menu_dj.m3").click() #테마장르 클릭
time.sleep(0.5)

tag_input = driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[1]/div[2]/div[2]/input")
tag_input.send_keys("상처")
driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[1]/div[2]/div[2]/button").click() # 검색 클릭

In [3]:
k = 0
dict = {}    # 전체 크롤링 데이터 dictionary로 저장

In [ ]:
for i in range(1,21):
    song_count_xpath = "/html/body/div/div[3]/div/div/div[4]/div/ul/li[" + str(i) + "]/div[2]/div[2]/span"
    try : 
        song_count = driver.find_element_by_xpath(song_count_xpath).text
        song_count = song_count.replace("곡","")
        song_count = song_count.replace(",","")
        playlist_kind = "/html/body/div/div[3]/div/div/div[4]/div/ul/li["+str(i)+"]/div[2]/div[1]/a[2]"
    except : 
        continue
    try : 
        driver.find_element_by_xpath(playlist_kind).click()
    except : 
        i = i + 1
        continue
    print(song_count)
    page_2 = False
    page_3 = False
    page_4 = False
    page_5 = False
    page_6 = False
    page_7 = False
    page_8 = False
    
    for j in range(1,int(song_count)):
        print(j)
        if(j >= 50 and j <100) :
            if(page_2 == False) :
                try : 
                    driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a").click()
                    page_2 = True
                    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
                except : 
                    continue
            try :
                more_info_list[j-50].click()
            except :
                continue
            
        elif(150>j>=100) :
            if(page_3 == False) :
                try : 
                    driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a[2]").click()
                    page_3 = True
                    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
                except : 
                    continue
            try :
                more_info_list[j-100].click()
            except : 
                continue
                
        elif(200>j>=150) :
            if(page_4 == False) :
                try : 
                    driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a[3]").click()
                    page_4 = True
                    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
                except : 
                    continue
            try :
                more_info_list[j-150].click()
            except : 
                continue
                
        elif(250>j>=200) :
            if(page_5 == False) :
                try : 
                    driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a[4]").click()
                    page_5 = True
                    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
                except : 
                    continue
            try :
                more_info_list[j-200].click()
            except : 
                continue
                
        elif(300>j>=250) :
            if(page_6 == False) :
                try : 
                    driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a[5]").click()
                    page_6 = True
                    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
                except : 
                    continue
            try :
                more_info_list[j-250].click()
            except : 
                continue
                
        elif(350>j>=300) :
            if(page_7 == False) :
                try : 
                    driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a[6]").click()
                    page_7 = True
                    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
                except : 
                    continue
            try :
                more_info_list[j-300].click()
            except : 
                continue
                
        elif(400>j>=350) :
            if(page_8 == False) :
                try : 
                    driver.find_element_by_xpath("/html/body/div/div[3]/div/div/div[4]/div[3]/div/span/a[7]").click()
                    page_8 = True
                    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
                except : 
                    continue
            try :
                more_info_list[j-350].click()
            except : 
                continue

        else :
            # 곡 한개씩 선택
            more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
            try : 
                more_info_list[j].click()            
            except : 
                continue
        
        try : 
            music_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
            time.sleep(0.5)
        
            # 제목 크롤링
            overlays = ".song_name"                                 
            tit = driver.find_element_by_css_selector(overlays)          # title
            title = tit.text

            # 가수 크롤링
            overlays = ".artist"                              
            nick = driver.find_element_by_css_selector(overlays)         # nickname
            artist_name = nick.text

            # 발매날짜 크롤링
            overlays = ".list"                                
            meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')  # datetime
            Album_date = meta_info[3]
        
            # 좋아요 수
            overlays = "d_like_count"
            like_count = driver.find_element_by_id(overlays)
            LIKE = like_count.text
        
            # 앨범이름 크롤링
            Album_name = meta_info[1]
        
            # 현재 곡 장르 크롤링
            Album_Genre = meta_info[5]
        
            # 더보기 버튼 클릭
            driver.find_element_by_css_selector(".button_more.arrow_d").click()
            time.sleep(0.5)
        
            # 가사 크롤링
            overlays = ".lyric.on"                                 
            contents = driver.find_elements_by_css_selector(overlays)    # contents
        
            content_list = []
            for content in contents:
                content_list.append(content.text)
 
            content_str = ' '.join(content_list)                         # content_str

            # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
            music_info['title'] = title
            music_info['artist'] = artist_name
            music_info['Album_date'] = Album_date
            music_info['LIKE'] = LIKE
            music_info['Album_name'] = Album_name
            music_info['Album_Genre'] = Album_Genre
            music_info['Lyric'] = content_str
            time.sleep(1)
            # 각각의 곡 정보는 dict라는 딕셔너리에 담기게 됩니다.
            dict[k] = music_info
            k = k + 1
            # 글 하나 크롤링 후 뒤로가기
            driver.back()  
    
     # 에러처리 : 에러나면 뒤로가기하고 다음 글(i+1)로 이동합니다.
        except:
            driver.back()
            i = i+1
            continue
        
    if(j==int(song_count)-1) :
        #  playlist 한 개 끝날때마다 pandas dataframe으로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        driver.back()
        if(page_2 == True) :
            driver.back()
        if(page_3 == True) : 
            driver.back()
        if(page_4 == True) : 
            driver.back()
        if(page_5 == True) : 
            driver.back()
        if(page_6 == True) : 
            driver.back()
        if(page_7 == True) : 
            driver.back()
        if(page_8 == True) : 
            driver.back()
        time.sleep(1)
        
# 저장하기
result_df.to_csv("Music_melon_wound2_tag.csv", encoding='utf-8-sig')
time.sleep(0.5)
driver.quit()

54
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
49
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
10
1
2
3
4
5
6
7
8
9
85
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
33
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [ ]:
result_df

In [11]:
data = pd.read_csv("Music_Melon.csv",encoding='utf-8')
data.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,That That (prod. & feat. SUGA of BTS),싸이 (PSY),2022.04.29,"61,873",싸다9,댄스,20 22\nPSY Coming back (이리 오너라)\n\nLong time n...
1,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),2022.04.05,"260,548",봄여름가을겨울 (Still Life),록/메탈,이듬해 질 녘 꽃 피는 봄 한여름 밤의 꿈\n가을 타 겨울 내릴 눈 1년 네 번 또...
2,2,LOVE DIVE,IVE (아이브),2022.04.05,"113,987",LOVE DIVE,댄스,네가 참 궁금해 그건 너도 마찬가지\n이거면 충분해 쫓고 쫓는 이런 놀이\n참을 수...
3,3,TOMBOY,(여자)아이들,2022.03.14,"160,448",I NEVER DIE,록/메탈,Look at you 넌 못 감당해 날\nYa took off hook\n기분은 C...
4,4,사랑인가 봐,멜로망스,2022.02.18,"104,535",사랑인가 봐 (사내맞선 OST 스페셜 트랙),"발라드, 국내드라마",너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는...


In [12]:
data_happy = pd.read_csv("Music_melon_happy_tag.csv",encoding = 'utf-8')
data_happy.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,Wish Now Was Later,Eli,2020.09.04,"5,552",Wish Now Was Later,POP,Time is not on our side\nnot in our favour\nGo...
1,1,Have It All,Jason Mraz,2018.08.10,"15,229",Know.,록/메탈,May you have our auspiciousness\nand causes of...
2,2,what are we,Virginia To Vegas,2020.02.28,"37,860",a constant state of improvement.,POP,I could be out with somebody new\nBut I'd only...
3,3,Where Were You In The Morning?,Shawn Mendes,2018.05.25,"48,528",Shawn Mendes,POP,You said I wanna get to know ya\nWhy you gotta...
4,4,Parties,Jake Miller,2017.06.16,"8,608",2:00am in LA,POP,I don't go to parties anymore\n'Cause I'm afra...


In [13]:
data_joy = pd.read_csv("Music_melon_joy_tag.csv",encoding = 'utf-8')
data_joy.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,"믿음이 없이는 (Feat. 김동욱, 강지은, 조서연, 조성범)",히즈윌 (HisWill),2017.10.24,"8,750",믿음이 필요해,"CCM, 국내CCM",주님 제 마음이 너무 둔해서\n주님을 볼 수 없습니다\n이 땅에 속하여\n이 땅만 ...
1,1,감사함으로,마커스,2006.03.01,"5,566",Our Song 2 - Face To Face,"CCM, 국내CCM",여호와를 즐거이 불러\n기쁨으로 주께 나가리\n여호와 하나님 난 주의 백성\n기르시...
2,2,주 나와 함께하시니 (시편23편),WELOVE,2018.08.28,"4,917",WELOVE ON THE RIVER,"CCM, 국내CCM",찬양 높이 계신 주께\n우리 맘을 열어\n높여 모든 영광 돌리세\n주 사랑하는 자\...
3,3,내 마음을 가득 채운,옹기장이,2009.06.10,"3,497",옹기장이 20주년 콘서트: 영원히 찬양드리리,"CCM, 국내CCM",내 마음을 가득 채운 주 향한 찬양과 사랑\n\n어떻게 표현할 수 있나\n\n수 많...
4,4,Amazing Grace,제이어스,2012.05.29,"2,354",Chesed (헤세드),"CCM, 국내CCM",나 같은 죄인 살리신\n주 은혜 놀라워\n잃었던 생명 찾았고\n광명을 얻었네\n큰 ...


In [14]:
data_angry = pd.read_csv("Music_melon_angry_tag.csv",encoding = 'utf-8')
data_angry.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,"business boy (Feat. 김농밀, BLOO)",허성현 (Huh!),2021.05.17,"13,356",business boy,랩/힙합,바닥에서 we go up here\nㅈㄴ 잘해 근데 No pills\n지갑 부풀어 ...
1,1,벌렁벌렁 (feat. Northfacegawd),Don Mills,2021.08.24,584,벌렁벌렁,랩/힙합,걸러 눈깔 돌아가는 놈들 전부 걸러\n벌렁 눈깔 안 돌아가는 거 보면 심장 벌렁\n...
2,2,날 (Feat. JTONG),지코 (ZICO),2015.11.03,"50,479",Boys and Girls,랩/힙합,I'll spit on ya like a king cobra\n한량들아 토껴 나는 ...
3,3,"Face ID (Feat. 기리보이, Sik-K, JUSTHIS)",에픽하이 (EPIK HIGH),2021.10.25,"8,747",Face ID,랩/힙합,"죽고 산 게 엊그제\n낙인 한두 개는 뭐, 풀옵션인데\n찍혀도 내 뜻대로면 no p..."
4,4,DAx4,사이먼 도미닉,2019.08.14,"17,876",DAx4,랩/힙합,댐벼 댐벼 다 DA DA DA DA DA\n나 vs 다 DA DA DA DA DA\...


In [15]:
data_anxiety = pd.read_csv("Music_melon_anxiety_tag.csv",encoding = 'utf-8')
data_anxiety.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,새벽 안개 (Remastered),조동진,2017.08.08,18,조동진 Remaster,포크/블루스,밤새 뒤척이던 이 혼란의 새벽\n그대 거짓 웃음과 내 짧은 입맞춤\n내게 남겨진 부...
1,1,그렇게 살아가는 것,허회경,2022.01.07,"14,711",그렇게 살아가는 것,"인디음악, 포크/블루스",가시 같은 말을 내뱉고\n날씨 같은 인생을 탓하고\n또 사랑 같은 말을 다시 내뱉는...
2,2,나아지지 않는 날 데리고 산다는 건,Mingginyu (밍기뉴),2022.02.14,"3,334",나아지지 않는 날 데리고 산다는 건,"발라드, 인디음악",난 모든 게 너무 지쳐서\n내려놓으려고 했는데\n왜 나는 나아지지가 않는 걸까\n오...
3,3,가시나무,시인과 촌장,1988,"2,403",시인과 촌장 3집 '숲',발라드,내속엔 내가 너무도 많아\n당신에 쉴곳 없네\n내속엔 헛된 바람들로\n당신에 편할곳...
4,4,Hidden Track,더더 (THETHE),2003.04.28,56,더더 4집,록/메탈,넌 어디에 있나 볼수가 없어 너무 지친 날 사랑으로\n죽여줘\n넌 어디에 있나 느낄...


In [16]:
data_anxiety2= pd.read_csv("Music_melon_anxiety2_tag.csv",encoding = 'utf-8')
data_anxiety2.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,지친 하루,"윤종신, 곽진언, 김필",2014.12.20,"85,275",2014 월간 윤종신 12월호,록/메탈,거기까지라고\n누군가 툭 한마디 던지면\n그렇지 하고 포기할 것 같아\n잘한 거라 ...
1,1,낯선 하루,마은진 (플레이백),2017.09.06,"2,380",병원선 OST Part.2,"발라드, 국내드라마",어제 걷던 길을\n다시 또 한걸음 걸어가고\n닿지 않는 꿈은 그 빛을 잃어가고\n고...
2,2,At The End,이창섭,2017.04.24,"62,214",Piece of BTOB Vol.1,발라드,새벽을 걷는다\n오늘 남은 모든 걸 흩날리듯\n밤공기 품은 채 어둡기만 한\n끝도 ...
3,3,오르막길,"정인, 윤종신",2012.05.30,"115,270",2012 월간 윤종신 6월호,발라드,이제부터 웃음기 사라질거야\n가파른 이 길을 좀 봐\n그래 오르기 전에\n미소를 기...
4,4,갈 수가 없어,벤,2017.11.28,"8,286",이번 생은 처음이라 OST,"발라드, 국내드라마",방 안을 한 가득\n채워놨었던 꿈들이\n한 장씩 의미를 잃어가서\n바래졌어\n지난 ...


In [17]:
data_sad = pd.read_csv("Music_melon_sad_tag.csv",encoding = 'utf-8')
data_sad.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,나비소녀 (퀸덤2 ver.),해와 달 (이달의 소녀 & Kep1er),2022.05.06,"4,028",<퀸덤2> 포지션 유닛 대결 Part.1-1,발라드,조그만 날갯짓 널 향한 이끌림\n나에게 따라오라 손짓한 것 같아서\n애절한 눈빛과 ...
1,1,"그날, 우리 (아홉수 우리들 X 류수정)",류수정,2022.05.07,"1,616","그날, 우리 (아홉수 우리들 X 류수정)",발라드,조용하게 기다리다 울어 난\n이제 와서 그래\n끝인 거 알면서\n오늘도 널 미워하다...
2,2,눈 떠보니 이별이더라,빅마마,2022.05.04,"3,532",눈 떠보니 이별이더라,발라드,분명 나는 꿈을 꾼 거다\n눈 떠보니 이별이더라\n휴대폰을 확인해 봤더니\n다시 봐...
3,3,내일이 빛날 테니까 (You will shine),"박보람, 인순이",2022.05.09,871,내일이 빛날 테니까 (You will shine),발라드,어른이 되는 게 이런 건가요\n걱정만 앞설 뿐이죠\n나름 다 괜찮다 생각했는데\n아...
4,4,오늘만은 (Hiding Words),려욱 (RYEOWOOK),2022.05.03,"1,811",A Wild Rose - The 3rd Mini Album,발라드,우리 둘의 처음을 떠올려보아요\n어둠 속의 빛이었던 그대\n어지러운 세상에 당신을 ...


In [18]:
data_wound = pd.read_csv("Music_melon_wound_tag.csv",encoding = 'utf-8')
data_wound.head()

,Unnamed: 0,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,0,소낙비,예빛,2020.07.29,"3,761",그대와 춤을,"인디음악, 포크/블루스",하얀 꽃이 피는 날엔\n어김없이 나를 찾아온다구요\n이 밤도 그믐달도 지는 날엔\n...
1,1,잠이 와,허튼,2021.04.30,13,몸의 기억,"인디음악, 록/메탈",잠이 와\n지금 잠을 자면\n오랜 잠을 잘 거야\n오래된 잠을 잘 거야\n잠이 와\...
2,2,"한강, 지하철",장희원,2020.07.08,"1,425","한강, 지하철",포크/블루스,덜컹 흔들리는 나를 태우고\n하루의 끝을 지나는 지하철 속에\n캄캄했던 창문들 뒤로...
3,3,Fine,다린,2021.09.26,"1,838",Fine,포크/블루스,아무 말 하지 않아도 돼\n그저 나란히 여기에 누워\n우리를 껴안는 침묵에\n물드는...
4,4,시시콜콜한 이야기,이소라,2004.12.10,"24,821",눈썹달,발라드,잠깐 일어나봐 깨워서 미안해\n나도 모르겠어\n윤오의 진짜 마음을\n같이 걸을때도\...


In [ ]:
data_wound2 = pd.read_csv("Music_melon_wound2_tag.csv",encoding = 'utf-8')
data_wound2.head()

In [ ]:
result_df = pd.concat([data,data_happy,data_joy,data_angry,data_anxiety,data_anxiety2,data_sad,data_wound,data_wound2])
result_df.head()

In [ ]:
result_df.to_csv("music_more.csv",encoding='utf-8-sig')
result_df = pd.DataFrame()
result_df = pd.read_csv("music_more.csv",encoding='utf-8')
result_df['Lyric'][0]

In [ ]:
from hanspell import spell_checker

for i in range(result_df.shape[0]) :
    try :
        result_df['Lyric'][i] = result_df['Lyric'][i].replace("\n","")
        result_df['Lyric'][i] = translator.translate(result_df['Lyric'][i],src="en",dest="ko").text
        spell_check = spell_checker.check(result_df['Lyric'][i])
        spell_check = spell_check.as_dict() # dictionary 형태로 저장
        if(spell_check['result'] == True) :
            result_df['Lyric'][i] = spell_check['checked']
    except :
        continue

In [ ]:
result_df.shape[0]

In [ ]:
lyric_sentiment_list = []

import joblib

tfidf_save_model = joblib.load('tfidf.pkl')
lgbm_clf_save_model = joblib.load('sentiment_analysis.pkl')

In [ ]:
for i in range(0,result_df.shape[0]):
    try :
        if result_df['Lyric'][i] != [] :
            lyric_sentiment = list(lgbm_clf_save_model.predict(tfidf_save_model.transform([result_df['Lyric'][i]])))
            lyric_sentiment_list.append(lyric_sentiment[0])
    except:
        continue

In [ ]:
lyric_sentiment_df = pd.DataFrame(lyric_sentiment_list)
lyric_sentiment_df.columns = ["label"]
lyric_sentiment_df

In [ ]:
Lyric_df = pd.merge(result_df,lyric_sentiment_df,left_index= True, right_index=True )
Lyric_df

In [ ]:
Lyric_df = pd.read_csv("music_more_list.csv",encoding='utf-8')

### 0 : 기쁨 1 : 행복 2 : 분노 3 : 불안 4 : 당황 5: 슬픔 6 : 상처

In [ ]:
Lyric_df['label'].value_counts()

In [ ]:
Lyric_df['label'].value_counts(1)*100

In [34]:
Liryc_df.to_csv("music_more_list.csv",encoding="utf-8-sig")